# Simularium Conversion Tutorial : Custom Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import (
    TrajectoryConverter, 
    TrajectoryData, 
    AgentData, 
    UnitData, 
    MetaData, 
    ModelMetaData,
    CameraData, 
    DisplayData,
    DISPLAY_TYPE,
)

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes spatiotemporal data from any source. It requires a `TrajectoryData` object as parameter ([see documentation](https://allen-cell-animated.github.io/simulariumio/simulariumio.data_objects.html#simulariumio.data_objects.trajectory_data.TrajectoryData)).

If you'd like to specify PDB or OBJ files or color for rendering an agent type, add a `DisplayData` object for that agent type, as shown below ([see documentation](https://allen-cell-animated.github.io/simulariumio/simulariumio.data_objects.html#module-simulariumio.data_objects.display_data)).


### Generate example data

To demonstrate using the custom converter, we'll first generate some random example data.

#### Only default agents:

In [2]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10

type_names = []
for t in range(total_steps):
    type_names.append([choice(ascii_uppercase) for i in range(n_agents)])
    
display_data = {}
unique_types = set([tn for frame in type_names for tn in frame])
for type_name in unique_types:
    display_data[type_name] = DisplayData(
        name=type_name,
        display_type=DISPLAY_TYPE.SPHERE,
    )
    
example_default_data = TrajectoryData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        camera_defaults=CameraData(
            position=np.array([10.0, 0.0, 200.0]),
            look_at_position=np.array([10.0, 0.0, 0.0]),
            fov_degrees=60.0,
        ),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/allen-cell-animated/simulariumio",
            source_code_license_url="https://github.com/allen-cell-animated/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    agent_data=AgentData(
        times=timestep * np.array(list(range(total_steps))),
        n_agents=np.array(total_steps * [n_agents]),
        viz_types=np.array(total_steps * [n_agents * [1000.0]]),  # default viz type = 1000
        unique_ids=np.array(total_steps * [list(range(n_agents))]),
        types=type_names,
        positions=np.random.uniform(size=(total_steps, n_agents, 3)) * box_size - box_size * 0.5,
        radii=(max_radius - min_radius) * np.random.uniform(size=(total_steps, n_agents)) + min_radius,
        display_data=display_data,
    ),
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

#### Fibers:

In [3]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10
points_per_fiber = 4

type_names = []
for t in range(total_steps):
    type_names.append([choice(ascii_uppercase) for i in range(n_agents)])
    
display_data = {}
unique_types = set([tn for frame in type_names for tn in frame])
for type_name in unique_types:
    display_data[type_name] = DisplayData(
        name=type_name,
        display_type=DISPLAY_TYPE.FIBER,
    )

example_fiber_data = TrajectoryData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        camera_defaults=CameraData(
            position=np.array([10.0, 0.0, 200.0]),
            look_at_position=np.array([10.0, 0.0, 0.0]),
            fov_degrees=60.0,
        ),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/allen-cell-animated/simulariumio",
            source_code_license_url="https://github.com/allen-cell-animated/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    agent_data=AgentData(
        times=timestep * np.array(list(range(total_steps))),
        n_agents=np.array(total_steps * [n_agents]),
        viz_types=np.array(total_steps * [n_agents * [1001.0]]), # fiber viz type = 1001
        unique_ids=np.array(total_steps * [list(range(n_agents))]),
        types=type_names,
        positions=np.zeros(shape=(total_steps, n_agents, 3)),
        radii=np.ones(shape=(total_steps, n_agents)),
        n_subpoints=points_per_fiber * np.ones(shape=(total_steps, n_agents)),
        subpoints=(
            box_size 
            * np.random.uniform(size=(total_steps, n_agents, points_per_fiber, 3))
            - box_size * 0.5
        ).reshape(total_steps, n_agents, 3 * points_per_fiber),
        display_data=display_data,
    ),
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

#### Metaballs:

In [4]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 3
timestep = 0.5
box_size = 100
n_agents = 2
metaballs_per_agent = 3 # this can vary per agent
values_per_metaball = 4 # position XYZ and radius for each metaball

type_names = [
    ["A", "B"],
    ["A", "B"],
    ["A", "B"],
]

metaball_subpoints = np.array([
    [ # timestep
        [ # agent
          # posX, posY, posZ, radius
            10,   12,   0,    2, # metaball 1
            12,   9,    0,    1, # metaball 2
            8,    9,    0,    1, # metaball 3
        ],
        [ # agent
            0,    10,   12,   1,
            0,    12,   9,    2,
            0,    8,    9,    2,
        ],
    ],
    [ # timestep
        [ # agent
            11,   13,    1,    2,
            13,   10,    1,    1,
            9,    10,    1,    1,
        ],
        [ # agent
            1,    10,   12,   1,
            1,    12,   9,    2,
            1,    8,    9,    2,
        ],
    ],
    [ # timestep
        [ # agent
            11,   13,    1,    2,
            13,   10,    1,    1,
            9,    10,    1,    1,
        ],
        [ # agent
            1,    10,   12,   1,
            1,    12,   9,    2,
            1,    8,    9,    2,
        ],
    ],
])

example_metaball_data = TrajectoryData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        camera_defaults=CameraData(
            position=np.array([10.0, 0.0, 200.0]),
            look_at_position=np.array([10.0, 0.0, 0.0]),
            fov_degrees=60.0,
        ),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/allen-cell-animated/simulariumio",
            source_code_license_url="https://github.com/allen-cell-animated/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    agent_data=AgentData(
        times=timestep * np.array(list(range(total_steps))),
        n_agents=np.array(total_steps * [n_agents]),
        viz_types=np.array(total_steps * [n_agents * [1000.0]]), # default viz type = 1000
        unique_ids=np.array(total_steps * [list(range(n_agents))]),
        types=type_names,
        positions=np.zeros(shape=(total_steps, n_agents, 3)),
        radii=np.ones(shape=(total_steps, n_agents)),
        n_subpoints=metaballs_per_agent * values_per_metaball * np.ones(shape=(total_steps, n_agents)),
        subpoints=metaball_subpoints,
        display_data={
            "A": DisplayData(
                name="A",
                display_type=DISPLAY_TYPE.METABALLS,
                color="#0080ff",
            ),
            "B": DisplayData(
                name="B",
                display_type=DISPLAY_TYPE.METABALLS,
                color="#666666",
            ),
        },
    ),
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_default_data` or `example_fiber_data` object, you can use the converter to generate the file at the given path:

In [5]:
TrajectoryConverter(example_fiber_data).save("example_fibers", binary=False)
TrajectoryConverter(example_default_data).save("example_default", binary=False)
TrajectoryConverter(example_metaball_data).save("example_metaballs", binary=False)

Converting Trajectory Data to JSON -------------
Writing JSON -------------
saved to example_fibers.simularium
Converting Trajectory Data to JSON -------------
Writing JSON -------------
saved to example_default.simularium
Converting Trajectory Data to JSON -------------
Writing JSON -------------
saved to example_metaballs.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.